# Morning

In [1]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *
import numpy as np
import csv
import os
import time

# ✅ Load YOLO model
model = YOLO('yolov8s.pt')

# ✅ Object classes
class_list = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "dining table", "toilet", "TV", "laptop",
    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# ✅ Initialize tracker
tracker = Tracker()
cap = cv2.VideoCapture('camp3_morning.MOV')

# ✅ CSV File Setup
csv_filename = "vehicles_summary.csv"

# ✅ Create CSV file if it doesn't exist
if not os.path.exists(csv_filename):
    with open(csv_filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Cars", "Buses", "Trucks", "Motorcycles"])

# ✅ Define reference lines
red_line_y = 385
blue_line_y = 415
offset = 7  # Allow slight variation due to tracking jitter

# ✅ Define vehicle classes to track
vehicle_classes = ["car", "bus", "truck", "motorcycle"]
vehicle_counts = {"car": 0, "bus": 0, "truck": 0, "motorcycle": 0}

# ✅ Store vehicle IDs that crossed the red line first
crossed_red = {}
counter_down = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1020, 500))

    # ✅ YOLO Detection
    results = model.predict(frame)
    a = results[0].boxes.data.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    # ✅ Store detections
    detections = []
    detected_vehicles = {}

    for _, row in px.iterrows():
        x1, y1, x2, y2 = int(row[0]), int(row[1]), int(row[2]), int(row[3])
        d = int(row[5])
        vehicle_type = class_list[d]

        if vehicle_type in vehicle_classes:
            detections.append([x1, y1, x2, y2])
            detected_vehicles[(x1, y1, x2, y2)] = vehicle_type

    # ✅ Ensure 'detections' is not empty
    bbox_id = tracker.update(np.array(detections)) if detections else []

    # ✅ Process ALL detected vehicles
    for bbox in bbox_id:
        x3, y3, x4, y4, vehicle_id = bbox
        cx, cy = (x3 + x4) // 2, (y3 + y4) // 2

        vehicle_type = detected_vehicles.get((x3, y3, x4, y4), "unknown")

        # ✅ Check if the vehicle crosses the RED line first
        if red_line_y - offset < cy < red_line_y + offset:
            crossed_red[vehicle_id] = (cy, vehicle_type)

        # ✅ Only consider vehicles that crossed red before reaching blue
        if vehicle_id in crossed_red and blue_line_y - offset < cy < blue_line_y + offset:
            _, v_type = crossed_red[vehicle_id]

            # ✅ Draw circle at the blue line for valid vehicles
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"{v_type}-{vehicle_id}", (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

            # ✅ Add to counter only once
            if vehicle_id not in counter_down:
                counter_down.append(vehicle_id)
                vehicle_counts[v_type] += 1

    # ✅ Draw reference lines
    cv2.line(frame, (260, red_line_y), (724, red_line_y), (0, 0, 255), 3)
    cv2.putText(frame, 'red line', (260, red_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.line(frame, (200, blue_line_y), (779, blue_line_y), (255, 0, 0), 3)
    cv2.putText(frame, 'blue line', (200, blue_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # ✅ Display vehicle count
    cv2.putText(frame, f"Cars: {vehicle_counts['car']}  Buses: {vehicle_counts['bus']}  Trucks: {vehicle_counts['truck']}  Motorcycles: {vehicle_counts['motorcycle']}",
                (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow("frames", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# ✅ Log the final vehicle count when the video ends
timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
number_of_lanes=2
# ✅ Calculate light duration using the given formula
light_duration = ((vehicle_counts["car"] * 2) + 
                  (vehicle_counts["motorcycle"] * 1.8) + 
                  ((vehicle_counts["bus"] + vehicle_counts["truck"]) * 3.5)) / number_of_lanes
with open(csv_filename, mode="a", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([timestamp, vehicle_counts["car"], vehicle_counts["bus"], vehicle_counts["truck"], vehicle_counts["motorcycle"], round(light_duration, 2)])

print(f"✅ Final vehicle count saved to CSV: {vehicle_counts}, Light Duration: {round(light_duration, 2)} sec")


cap.release()
cv2.destroyAllWindows()



0: 320x640 6 cars, 3 motorcycles, 2 trucks, 288.1ms
Speed: 5.5ms preprocess, 288.1ms inference, 3.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 6 cars, 3 motorcycles, 1 bus, 1 truck, 245.9ms
Speed: 6.5ms preprocess, 245.9ms inference, 1.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 7 cars, 3 motorcycles, 1 bus, 1 truck, 247.1ms
Speed: 2.9ms preprocess, 247.1ms inference, 1.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 6 cars, 2 motorcycles, 1 bus, 2 trucks, 267.6ms
Speed: 4.5ms preprocess, 267.6ms inference, 2.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 5 cars, 2 motorcycles, 1 bus, 1 truck, 293.5ms
Speed: 96.8ms preprocess, 293.5ms inference, 3.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 5 cars, 2 motorcycles, 1 bus, 1 truck, 239.4ms
Speed: 4.3ms preprocess, 239.4ms inference, 2.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x64

KeyboardInterrupt: 

# Noon

In [2]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *
import numpy as np
import csv
import os
import time

# ✅ Load YOLO model
model = YOLO('yolov8s.pt')

# ✅ Object classes
class_list = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "dining table", "toilet", "TV", "laptop",
    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# ✅ Initialize tracker
tracker = Tracker()
cap = cv2.VideoCapture('IMG_8980.MOV')

# ✅ CSV File Setup
csv_filename = "vehicles_summary.csv"

# ✅ Create CSV file if it doesn't exist
if not os.path.exists(csv_filename):
    with open(csv_filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Cars", "Buses", "Trucks", "Motorcycles"])

# ✅ Define reference lines
red_line_y = 338
blue_line_y = 380
offset = 7  # Allow slight variation due to tracking jitter

# ✅ Define vehicle classes to track
vehicle_classes = ["car", "bus", "truck", "motorcycle"]
vehicle_counts = {"car": 0, "bus": 0, "truck": 0, "motorcycle": 0}

# ✅ Store vehicle IDs that crossed the red line first
crossed_red = {}
counter_down = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1020, 500))

    # ✅ YOLO Detection
    results = model.predict(frame)
    a = results[0].boxes.data.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    # ✅ Store detections
    detections = []
    detected_vehicles = {}

    for _, row in px.iterrows():
        x1, y1, x2, y2 = int(row[0]), int(row[1]), int(row[2]), int(row[3])
        d = int(row[5])
        vehicle_type = class_list[d]

        if vehicle_type in vehicle_classes:
            detections.append([x1, y1, x2, y2])
            detected_vehicles[(x1, y1, x2, y2)] = vehicle_type

    # ✅ Ensure 'detections' is not empty
    bbox_id = tracker.update(np.array(detections)) if detections else []

    # ✅ Process ALL detected vehicles
    for bbox in bbox_id:
        x3, y3, x4, y4, vehicle_id = bbox
        cx, cy = (x3 + x4) // 2, (y3 + y4) // 2

        vehicle_type = detected_vehicles.get((x3, y3, x4, y4), "unknown")

        # ✅ Check if the vehicle crosses the RED line first
        if red_line_y - offset < cy < red_line_y + offset:
            crossed_red[vehicle_id] = (cy, vehicle_type)

        # ✅ Only consider vehicles that crossed red before reaching blue
        if vehicle_id in crossed_red and blue_line_y - offset < cy < blue_line_y + offset:
            _, v_type = crossed_red[vehicle_id]

            # ✅ Draw circle at the blue line for valid vehicles
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"{v_type}-{vehicle_id}", (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

            # ✅ Add to counter only once
            if vehicle_id not in counter_down:
                counter_down.append(vehicle_id)
                vehicle_counts[v_type] += 1

    # ✅ Draw reference lines
    cv2.line(frame, (260, red_line_y), (724, red_line_y), (0, 0, 255), 3)
    cv2.putText(frame, 'red line', (260, red_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.line(frame, (200, blue_line_y), (779, blue_line_y), (255, 0, 0), 3)
    cv2.putText(frame, 'blue line', (200, blue_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # ✅ Display vehicle count
    cv2.putText(frame, f"Cars: {vehicle_counts['car']}  Buses: {vehicle_counts['bus']}  Trucks: {vehicle_counts['truck']}  Motorcycles: {vehicle_counts['motorcycle']}",
                (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow("frames", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# ✅ Log the final vehicle count when the video ends
timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
number_of_lanes=2
# ✅ Calculate light duration using the given formula
light_duration = ((vehicle_counts["car"] * 2) + 
                  (vehicle_counts["motorcycle"] * 1.8) + 
                  ((vehicle_counts["bus"] + vehicle_counts["truck"]) * 3.5)) / number_of_lanes
with open(csv_filename, mode="a", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([timestamp, vehicle_counts["car"], vehicle_counts["bus"], vehicle_counts["truck"], vehicle_counts["motorcycle"], round(light_duration, 2)])

print(f"✅ Final vehicle count saved to CSV: {vehicle_counts}, Light Duration: {round(light_duration, 2)} sec")


cap.release()
cv2.destroyAllWindows()



0: 320x640 1 person, 7 cars, 1 motorcycle, 2 trucks, 406.0ms
Speed: 5.8ms preprocess, 406.0ms inference, 2.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 7 cars, 2 motorcycles, 1 truck, 227.5ms
Speed: 4.3ms preprocess, 227.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 7 cars, 2 motorcycles, 221.5ms
Speed: 5.4ms preprocess, 221.5ms inference, 2.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 6 cars, 2 motorcycles, 1 truck, 228.2ms
Speed: 3.3ms preprocess, 228.2ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 8 cars, 2 motorcycles, 1 bus, 1 truck, 243.0ms
Speed: 4.6ms preprocess, 243.0ms inference, 2.9ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 7 cars, 1 motorcycle, 1 truck, 268.4ms
Speed: 5.6ms preprocess, 268.4ms inference, 3.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 8 cars, 1 motorcycle,

KeyboardInterrupt: 

# Rain

In [3]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *
import numpy as np
import csv
import os
import time

# ✅ Load YOLO model
model = YOLO('yolov8s.pt')

# ✅ Object classes
class_list = [
    "person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake",
    "chair", "couch", "potted plant", "bed", "dining table", "toilet", "TV", "laptop",
    "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink",
    "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

# ✅ Initialize tracker
tracker = Tracker()
cap = cv2.VideoCapture('camp3_rain_noon.mp4')

# ✅ CSV File Setup
csv_filename = "vehicles_summary.csv"

# ✅ Create CSV file if it doesn't exist
if not os.path.exists(csv_filename):
    with open(csv_filename, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Timestamp", "Cars", "Buses", "Trucks", "Motorcycles"])

# ✅ Define reference lines
red_line_y = 340
blue_line_y = 370
offset = 7  # Allow slight variation due to tracking jitter

# ✅ Define vehicle classes to track
vehicle_classes = ["car", "bus", "truck", "motorcycle"]
vehicle_counts = {"car": 0, "bus": 0, "truck": 0, "motorcycle": 0}

# ✅ Store vehicle IDs that crossed the red line first
crossed_red = {}
counter_down = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.resize(frame, (1020, 500))

    # ✅ YOLO Detection
    results = model.predict(frame)
    a = results[0].boxes.data.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    # ✅ Store detections
    detections = []
    detected_vehicles = {}

    for _, row in px.iterrows():
        x1, y1, x2, y2 = int(row[0]), int(row[1]), int(row[2]), int(row[3])
        d = int(row[5])
        vehicle_type = class_list[d]

        if vehicle_type in vehicle_classes:
            detections.append([x1, y1, x2, y2])
            detected_vehicles[(x1, y1, x2, y2)] = vehicle_type

    # ✅ Ensure 'detections' is not empty
    bbox_id = tracker.update(np.array(detections)) if detections else []

    # ✅ Process ALL detected vehicles
    for bbox in bbox_id:
        x3, y3, x4, y4, vehicle_id = bbox
        cx, cy = (x3 + x4) // 2, (y3 + y4) // 2

        vehicle_type = detected_vehicles.get((x3, y3, x4, y4), "unknown")

        # ✅ Check if the vehicle crosses the RED line first
        if red_line_y - offset < cy < red_line_y + offset:
            crossed_red[vehicle_id] = (cy, vehicle_type)

        # ✅ Only consider vehicles that crossed red before reaching blue
        if vehicle_id in crossed_red and blue_line_y - offset < cy < blue_line_y + offset:
            _, v_type = crossed_red[vehicle_id]

            # ✅ Draw circle at the blue line for valid vehicles
            cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
            cv2.putText(frame, f"{v_type}-{vehicle_id}", (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

            # ✅ Add to counter only once
            if vehicle_id not in counter_down:
                counter_down.append(vehicle_id)
                vehicle_counts[v_type] += 1

    # ✅ Draw reference lines
    cv2.line(frame, (260, red_line_y), (724, red_line_y), (0, 0, 255), 3)
    cv2.putText(frame, 'red line', (260, red_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    cv2.line(frame, (200, blue_line_y), (779, blue_line_y), (255, 0, 0), 3)
    cv2.putText(frame, 'blue line', (200, blue_line_y), cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)

    # ✅ Display vehicle count
    cv2.putText(frame, f"Cars: {vehicle_counts['car']}  Buses: {vehicle_counts['bus']}  Trucks: {vehicle_counts['truck']}  Motorcycles: {vehicle_counts['motorcycle']}",
                (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

    cv2.imshow("frames", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

# ✅ Log the final vehicle count when the video ends
timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
number_of_lanes=2
# ✅ Calculate light duration using the given formula
light_duration = ((vehicle_counts["car"] * 2) + 
                  (vehicle_counts["motorcycle"] * 1.8) + 
                  ((vehicle_counts["bus"] + vehicle_counts["truck"]) * 3.5)) / number_of_lanes
with open(csv_filename, mode="a", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([timestamp, vehicle_counts["car"], vehicle_counts["bus"], vehicle_counts["truck"], vehicle_counts["motorcycle"], round(light_duration, 2)])

print(f"✅ Final vehicle count saved to CSV: {vehicle_counts}, Light Duration: {round(light_duration, 2)} sec")


cap.release()
cv2.destroyAllWindows()



0: 320x640 4 persons, 6 cars, 3 motorcycles, 3 trucks, 237.9ms
Speed: 5.4ms preprocess, 237.9ms inference, 2.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 4 cars, 3 motorcycles, 3 trucks, 1 umbrella, 240.7ms
Speed: 6.1ms preprocess, 240.7ms inference, 2.8ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 4 cars, 3 motorcycles, 3 trucks, 1 umbrella, 239.6ms
Speed: 3.7ms preprocess, 239.6ms inference, 2.6ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 6 cars, 3 motorcycles, 1 bus, 3 trucks, 1 umbrella, 225.2ms
Speed: 2.9ms preprocess, 225.2ms inference, 2.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 4 persons, 6 cars, 3 motorcycles, 1 bus, 2 trucks, 1 umbrella, 218.0ms
Speed: 3.6ms preprocess, 218.0ms inference, 2.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 6 cars, 2 motorcycles, 3 trucks, 1 umbrella, 228.0ms
Speed: 2.8ms preprocess, 228.0ms inference, 2.3ms po

KeyboardInterrupt: 